In [1]:
import cv2
import numpy as np
import pygame
import time
from scipy.spatial import distance as dist
import mediapipe as mp
import math  # Import math module for angle calculations

# Initialize mediapipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5)

# Initialize Pygame for the alert sounds
pygame.mixer.init()
primary_alert = pygame.mixer.Sound("primary_alert.wav")  # Primary alert sound
secondary_alert = pygame.mixer.Sound("secondary_alert.wav")  # Secondary alert sound

# Eye landmarks for EAR calculations
LEFT_EYE = [362, 385, 387, 263, 373, 380]
RIGHT_EYE = [33, 160, 158, 133, 153, 144]

# Thresholds and counters
EAR_THRESHOLD = 0.2
CLOSED_EYE_DURATION_THRESHOLD = 0.75  # Seconds
NORMAL_TILT_THRESHOLD = 6.0  # ±6 degrees for normal tilt
IMPAIRED_TILT_THRESHOLD = 20.0  # ±20 degrees indicates impairment

# Initialize variables
blink_count = 0
start_drowsy_time = None
is_drowsy = False
alert_count = 0
blink_rate = 0
start_time = time.time()
blink_interval = 1  # Calculate blink rate every 60 seconds

# Function to calculate Eye Aspect Ratio (EAR)
def calculate_ear(eye_landmarks):
    vertical_1 = dist.euclidean(eye_landmarks[1], eye_landmarks[5])
    vertical_2 = dist.euclidean(eye_landmarks[2], eye_landmarks[4])
    horizontal = dist.euclidean(eye_landmarks[0], eye_landmarks[3])
    ear = (vertical_1 + vertical_2) / (2.0 * horizontal)
    return ear

# Function to calculate head tilt angle in degrees
def calculate_head_tilt(face_landmarks):
    left_eye = np.array([face_landmarks.landmark[33].x, face_landmarks.landmark[33].y])
    right_eye = np.array([face_landmarks.landmark[263].x, face_landmarks.landmark[263].y])
    dx = right_eye[0] - left_eye[0]
    dy = right_eye[1] - left_eye[1]
    angle = math.degrees(math.atan2(dy, dx))
    return angle

# Start capturing video
cap = cv2.VideoCapture(0)

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        print("Failed to capture image")
        break
    
    # Convert the BGR frame to RGB before processing
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Get the face landmarks
    results = face_mesh.process(rgb_frame)
    
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Extract eye landmarks
            left_eye = [(face_landmarks.landmark[i].x * frame.shape[1], face_landmarks.landmark[i].y * frame.shape[0]) for i in LEFT_EYE]
            right_eye = [(face_landmarks.landmark[i].x * frame.shape[1], face_landmarks.landmark[i].y * frame.shape[0]) for i in RIGHT_EYE]

            # Calculate EAR for both eyes
            left_ear = calculate_ear(left_eye)
            right_ear = calculate_ear(right_eye)
            avg_ear = (left_ear + right_ear) / 2.0
            
            # Calculate head tilt angle
            head_tilt_angle = calculate_head_tilt(face_landmarks)

            # Blink detection based on EAR
            if avg_ear < EAR_THRESHOLD or abs(head_tilt_angle) > IMPAIRED_TILT_THRESHOLD:
                if start_drowsy_time is None:
                    start_drowsy_time = time.time()
                elif time.time() - start_drowsy_time >= CLOSED_EYE_DURATION_THRESHOLD:
                    # Trigger alert if eyes are closed for a prolonged duration
                    if not is_drowsy:
                        primary_alert.play()  # Play primary alert sound
                        alert_count += 1  # Increment alert count
                        is_drowsy = True
                    cv2.putText(frame, "DROWSY! Wake up!", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 3)
                    
                    # Check if alert count exceeds 4
                    if alert_count > 4:
                        secondary_alert.play()  # Play secondary alert sound
                        alert_count = 0  # Reset alert count
            else:
                if start_drowsy_time is not None:
                    blink_count += 1
                start_drowsy_time = None
                if is_drowsy:
                    is_drowsy = False  # Reset drowsy state

            # Calculate blink rate every specified interval
            elapsed_time = time.time() - start_time
            if elapsed_time >= blink_interval:
                blink_rate = blink_count / blink_interval
                blink_count = 0  # Reset blink count
                start_time = time.time()  # Reset start time

            # Display blink rate on the screen
            cv2.putText(frame, f"Blink Rate: {blink_rate:.1f} blinks/sec", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

            # Draw eye landmarks for visualization
            for point in left_eye + right_eye:
                cv2.circle(frame, (int(point[0]), int(point[1])), 2, (0, 255, 0), -1)

            # Display head tilt angle for visualization
            cv2.putText(frame, f"Head Tilt: {head_tilt_angle:.1f} degrees", (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)

    # Display the frame
    cv2.imshow('Drowsiness Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
cv2.destroyAllWindows()
pygame.quit()

pygame 2.6.1 (SDL 2.28.4, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html
